# Bike_sharing

# Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(y)
Y[:5]

# 2, Regression- np.sqrt(mse)


In [1]:
import pandas as pd
DF = pd.read_csv('https://raw.githubusercontent.com/rusita-ai/pyData/master/Bike_Sharing_Demand.csv')
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [2]:
DF.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
X = DF[['season','holiday','workingday','weather','temp','humidity','windspeed']]
y = DF['count']

# scaling

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
DF[['season','holiday','workingday','weather','temp','humidity','windspeed']] = scaler.fit_transform(X)

## 1, Multivariate Regression

### 모델링

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

In [7]:
def multiReg(model, X, y, dg):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X,y, stratify =y,test_size= 0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  MR = LinearRegression(normalize=True, n_jobs=-1)
  MR.fit(PX_tr, y_train)

  # evaluate
  MR_mse = np.sqrt(mean_squared_error(y_test, MR.predict(PX_te)))

  return {'model':model, 'degree':dg, 'mse':MR_mse}

### Hyperparameter(degree)

In [ ]:
DF_res = pd.DataFrame(columns=['model', 'alpha','degree',  'n_estimators', 'learning_rate', 'max_features', 'max_depth', 'mse'])

In [94]:
%%time
MR_mse = pd.DataFrame(columns=['model','degree','mse'])
a =[1,2,5,7]
index= 1

for i in a:
  model = 'MR-'+str(index)
  index += 1
  res = multiReg(model, X, y, dg=i)
  print(res)
  MR_mse = MR_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'MR-1', 'degree': 1, 'mse': 156.03395296591316}
{'model': 'MR-2', 'degree': 2, 'mse': 151.93139856165664}
{'model': 'MR-3', 'degree': 5, 'mse': 1295827594544.5159}
{'model': 'MR-4', 'degree': 7, 'mse': 363427590107.84955}
CPU times: user 52.6 s, sys: 2.53 s, total: 55.1 s
Wall time: 28.6 s


## 2, Ridge Regression

### 모델링

In [12]:
from sklearn.linear_model import Ridge

def ridge(model, X, y, dg, ap=0.3):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  RG = Ridge(normalize=True, alpha=ap, solver='cholesky')
  RG.fit(PX_tr, y_train)

  RG_mse = np.sqrt(mean_squared_error(y_test, RG.predict(PX_te)))

  return {'model':model, 'degree':dg, 'alpha':ap, 'mse':RG_mse}

### Hyperparameter(degree)

In [13]:
%%time
RG_mse = pd.DataFrame(columns=['model','degree','alpha','mse'])
a = [1,2,3,4]
index = 1

for i in a:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RG-1', 'degree': 1, 'alpha': 0.3, 'mse': 157.08589772713447}
{'model': 'RG-2', 'degree': 2, 'alpha': 0.3, 'mse': 155.31154849885817}
{'model': 'RG-3', 'degree': 3, 'alpha': 0.3, 'mse': 154.28456797646976}
{'model': 'RG-4', 'degree': 4, 'alpha': 0.3, 'mse': 153.33822329027635}


In [14]:
%%time
for i in [5,6,7]:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RG-5', 'degree': 5, 'alpha': 0.3, 'mse': 152.42341799563565}
{'model': 'RG-6', 'degree': 6, 'alpha': 0.3, 'mse': 151.68938013839383}
{'model': 'RG-7', 'degree': 7, 'alpha': 0.3, 'mse': 151.1928425035353}


In [15]:
%%time
for i in [8,9,10]:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RG-8', 'degree': 8, 'alpha': 0.3, 'mse': 150.91523216210038}
{'model': 'RG-9', 'degree': 9, 'alpha': 0.3, 'mse': 150.85883200355232}
{'model': 'RG-10', 'degree': 10, 'alpha': 0.3, 'mse': 151.7777399153868}


In [17]:
RG_mse.sort_values(by='mse')

,model,degree,alpha,mse
8,RG-9,9,0.3,150.858832
7,RG-8,8,0.3,150.915232
6,RG-7,7,0.3,151.192843
5,RG-6,6,0.3,151.689380
9,RG-10,10,0.3,151.777740
4,RG-5,5,0.3,152.423418
3,RG-4,4,0.3,153.338223
2,RG-3,3,0.3,154.284568
1,RG-2,2,0.3,155.311548
0,RG-1,1,0.3,157.085898


### Hyperparameter(alpha)

In [18]:
%%time
a = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9]

for i in a :
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=9 , ap=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RG-11', 'degree': 9, 'alpha': 0.01, 'mse': 179.7782994897138}
{'model': 'RG-12', 'degree': 9, 'alpha': 0.05, 'mse': 154.93449616144116}
{'model': 'RG-13', 'degree': 9, 'alpha': 0.1, 'mse': 151.53889800742218}
{'model': 'RG-14', 'degree': 9, 'alpha': 0.3, 'mse': 150.85883200355232}
{'model': 'RG-15', 'degree': 9, 'alpha': 0.5, 'mse': 151.29084786348537}
{'model': 'RG-16', 'degree': 9, 'alpha': 0.7, 'mse': 151.70487589594217}
{'model': 'RG-17', 'degree': 9, 'alpha': 0.9, 'mse': 152.07164146811968}


In [22]:
RG_mse.sort_values(by='mse').head(3)

,model,degree,alpha,mse
8,RG-9,9,0.3,150.858832
13,RG-14,9,0.3,150.858832
7,RG-8,8,0.3,150.915232


## 3, Lasso

### 모델링

In [23]:
from sklearn.linear_model import Lasso

def lasso(model, X, y, dg, ap=0.3):
  # spint
  X_train, X_test, y_train ,y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  LS = Lasso(normalize=True, alpha=ap)
  LS.fit(PX_tr, y_train)

  # mse
  LS_mse = np.sqrt(mean_squared_error(y_test, LS.predict(PX_te)))  

  return {'model':model, 'degree':dg, 'alpha':ap, 'mse':LS_mse}

### Hyperparameter(degree)

In [26]:
%%time
LS_mse = pd.DataFrame(columns=['model','degree','alpha','mse'])
a = [1,2,3,4,5,6,7,8,9,10]
b = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]
index = 1

for i in a :
  for j in b:
    model = 'LS-'+str(index)
    index += 1
    res = lasso(model, X, y, dg=i, ap=j)
    print(res)
    LS_mse = LS_mse.append(res, ignore_index=True)
    DF_res = DF_res.append(res, ignore_index=True)

{'model': 'LS-1', 'degree': 1, 'alpha': 0.01, 'mse': 156.03483114148662}
{'model': 'LS-2', 'degree': 1, 'alpha': 0.1, 'mse': 156.67968077801606}
{'model': 'LS-3', 'degree': 1, 'alpha': 0.3, 'mse': 161.47174247042977}
{'model': 'LS-4', 'degree': 1, 'alpha': 0.5, 'mse': 168.03552338148305}
{'model': 'LS-5', 'degree': 1, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-6', 'degree': 1, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-7', 'degree': 2, 'alpha': 0.01, 'mse': 152.7828900990057}
{'model': 'LS-8', 'degree': 2, 'alpha': 0.1, 'mse': 156.21371823141956}
{'model': 'LS-9', 'degree': 2, 'alpha': 0.3, 'mse': 160.77439051265412}
{'model': 'LS-10', 'degree': 2, 'alpha': 0.5, 'mse': 167.8895773520336}
{'model': 'LS-11', 'degree': 2, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-12', 'degree': 2, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-13', 'degree': 3, 'alpha': 0.01, 'mse': 151.8050696992396}
{'model': 'LS-14', 'degree': 3, 'alpha': 0.1, 'mse': 155.665

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 55053.87012270093, tolerance: 25086.59172675853
  positive)


{'model': 'LS-25', 'degree': 5, 'alpha': 0.01, 'mse': 151.02530671713856}
{'model': 'LS-26', 'degree': 5, 'alpha': 0.1, 'mse': 155.07752679080843}
{'model': 'LS-27', 'degree': 5, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-28', 'degree': 5, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-29', 'degree': 5, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-30', 'degree': 5, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-31', 'degree': 6, 'alpha': 0.01, 'mse': 150.73470858590477}
{'model': 'LS-32', 'degree': 6, 'alpha': 0.1, 'mse': 154.75922772931125}
{'model': 'LS-33', 'degree': 6, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-34', 'degree': 6, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-35', 'degree': 6, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-36', 'degree': 6, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-37', 'degree': 7, 'alpha': 0.01, 'mse': 150.50192402700253}
{'model': 'LS-38', 'degree': 7, 'alpha': 0.1, 'm

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 102433.59030893445, tolerance: 25086.59172675853
  positive)


{'model': 'LS-55', 'degree': 10, 'alpha': 0.01, 'mse': 150.72288527814004}
{'model': 'LS-56', 'degree': 10, 'alpha': 0.1, 'mse': 154.38258928925845}
{'model': 'LS-57', 'degree': 10, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-58', 'degree': 10, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-59', 'degree': 10, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-60', 'degree': 10, 'alpha': 0.9, 'mse': 180.42555843739754}


In [34]:
LS_mse.sort_values(by='mse').head(3)

,model,degree,alpha,mse
42,LS-43,8,0.01,150.441037
36,LS-37,7,0.01,150.501924
48,LS-49,9,0.01,150.567032


## 4, Decision Tree

### 모델링

In [32]:
from sklearn.tree import DecisionTreeRegressor

def decisionTree(model, X, y, md):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)

  # modeling
  DT = DecisionTreeRegressor(max_depth=md, criterion='mse')
  DT.fit(X_train, y_train)

  DT_mse = np.sqrt(mean_squared_error(y_test, DT.predict(X_test)))

  return {'model':model, 'max_depth':md, 'mse':DT_mse}

### Hyperparameter(max_depth)

In [33]:
DT_mse = pd.DataFrame(columns=['model','max_depth','mse'])
a = [2,3,5,7,9]
index = 1

for i in a:
  model = 'DT-'+str(index)
  index += 1
  res = decisionTree(model, X, y, md=i)
  print(res)
  DT_mse = DT_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'DT-1', 'max_depth': 2, 'mse': 160.74168249273998}
{'model': 'DT-2', 'max_depth': 3, 'mse': 156.0859197046373}
{'model': 'DT-3', 'max_depth': 5, 'mse': 151.6448906761661}
{'model': 'DT-4', 'max_depth': 7, 'mse': 149.83337137828704}
{'model': 'DT-5', 'max_depth': 9, 'mse': 154.74630523430213}


In [60]:
a = [11,13,15]

for i in a:
  model = 'DT-'+str(index)
  index += 1
  res = decisionTree(model, X, y, md=i)
  print(res)
  DT_mse = DT_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'DT-6', 'max_depth': 11, 'mse': 163.44963903391937}
{'model': 'DT-7', 'max_depth': 13, 'mse': 177.58821525817675}
{'model': 'DT-8', 'max_depth': 15, 'mse': 188.6741866702988}


## 5, Random Forest

### 모델링

In [58]:
from sklearn.ensemble import RandomForestRegressor

def randomForest(model, X, y, n_est, md=None, mf='auto', visualFeatures=False):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)
  
  # modeling
  RF = RandomForestRegressor(n_estimators=n_est, max_depth=md, max_features=mf, criterion='mse', n_jobs=-1, random_state=2045)
  RF.fit(X_train, y_train)

  # mse
  RF_mse = np.sqrt(mean_squared_error(y_test, RF.predict(X_test)))

  if visualFeatures:
    sns.barplot(RF.feature_importances_, X.columns)
    plt.title('random forest feature importances')
    plt.show()

  return {'model':model, 'n_estimators':n_est, 'max_depth':md, 'max_features':mf, 'mse':RF_mse}

### Hyperparameter(n_est)

In [59]:
%%time
RF_mse = pd.DataFrame(columns=['model','n_estimators','max_depth','max_features','mse'])
a = [50, 100, 200, 300, 500]
index = 1

for i in a:
  model = 'RF-'+str(index)
  index += 1
  res = randomForest(model, X, y, n_est=i)
  print(res)
  RF_mse = RF_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RF-1', 'n_estimators': 50, 'max_depth': None, 'max_features': 'auto', 'mse': 154.64987069836573}
{'model': 'RF-2', 'n_estimators': 100, 'max_depth': None, 'max_features': 'auto', 'mse': 154.02957040508295}
{'model': 'RF-3', 'n_estimators': 200, 'max_depth': None, 'max_features': 'auto', 'mse': 154.05750149449668}
{'model': 'RF-4', 'n_estimators': 300, 'max_depth': None, 'max_features': 'auto', 'mse': 154.07444152465322}
{'model': 'RF-5', 'n_estimators': 500, 'max_depth': None, 'max_features': 'auto', 'mse': 154.0279566544933}
CPU times: user 27.7 s, sys: 530 ms, total: 28.2 s
Wall time: 15.4 s


In [61]:
%%time
a = [700, 1000]

for i in a:
  model = 'RF-'+str(index)
  index += 1
  res = randomForest(model, X, y, n_est=i)
  print(res)
  RF_mse = RF_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RF-9', 'n_estimators': 700, 'max_depth': None, 'max_features': 'auto', 'mse': 153.975507376043}
{'model': 'RF-10', 'n_estimators': 1000, 'max_depth': None, 'max_features': 'auto', 'mse': 153.94370952615776}
CPU times: user 40.7 s, sys: 1.22 s, total: 41.9 s
Wall time: 22.4 s


In [62]:
RF_mse.sort_values(by='mse')

,model,n_estimators,max_depth,max_features,mse
6,RF-10,1000,None,auto,153.943710
5,RF-9,700,None,auto,153.975507
4,RF-5,500,None,auto,154.027957
1,RF-2,100,None,auto,154.029570
2,RF-3,200,None,auto,154.057501
3,RF-4,300,None,auto,154.074442
0,RF-1,50,None,auto,154.649871


### Hyperparameter(max_depth)

In [63]:
a = [2,3,5,7,9,11,13]

for i in a:
  model = 'RF-'+str(index)
  index += 1
  res = randomForest(model, X, y, n_est=1000, md=i)
  print(res)
  RF_mse = RF_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RF-11', 'n_estimators': 1000, 'max_depth': 2, 'max_features': 'auto', 'mse': 158.86813260977598}
{'model': 'RF-12', 'n_estimators': 1000, 'max_depth': 3, 'max_features': 'auto', 'mse': 154.4301321404243}
{'model': 'RF-13', 'n_estimators': 1000, 'max_depth': 5, 'max_features': 'auto', 'mse': 149.01586994237647}
{'model': 'RF-14', 'n_estimators': 1000, 'max_depth': 7, 'max_features': 'auto', 'mse': 146.14761010242879}
{'model': 'RF-15', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 'auto', 'mse': 144.75054862079725}
{'model': 'RF-16', 'n_estimators': 1000, 'max_depth': 11, 'max_features': 'auto', 'mse': 145.27384066188765}
{'model': 'RF-17', 'n_estimators': 1000, 'max_depth': 13, 'max_features': 'auto', 'mse': 147.73353575828824}


### Hyperparameter(max_features)

In [65]:
a = [1,2,3,5,7]

for i in a :
  model = 'RF-'+str(index)
  index += 1
  res = randomForest(model, X, y, n_est=1000, md=9, mf=i)
  print(res)
  RF_mse = RF_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'RF-19', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 1, 'mse': 147.5767423467687}
{'model': 'RF-20', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 2, 'mse': 145.99074787354164}
{'model': 'RF-21', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 3, 'mse': 145.10943573016505}
{'model': 'RF-22', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 5, 'mse': 144.5981891250126}
{'model': 'RF-23', 'n_estimators': 1000, 'max_depth': 9, 'max_features': 7, 'mse': 144.75054862079725}


In [66]:
RF_mse.sort_values(by='mse')

,model,n_estimators,max_depth,max_features,mse
17,RF-22,1000,9,5,144.598189
18,RF-23,1000,9,7,144.750549
11,RF-15,1000,9,auto,144.750549
16,RF-21,1000,9,3,145.109436
12,RF-16,1000,11,auto,145.273841
15,RF-20,1000,9,2,145.990748
10,RF-14,1000,7,auto,146.147610
14,RF-19,1000,9,1,147.576742
13,RF-17,1000,13,auto,147.733536
9,RF-13,1000,5,auto,149.015870


## 6, Gradient Boosting

### 모델링

In [73]:
from sklearn.ensemble import GradientBoostingRegressor

def gradient(model,X,y, n_est, lr=0.0001, mf='auto', md=None, visualFeatures=False):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)

  # modeling
  GB = GradientBoostingRegressor(loss='ls', n_estimators=n_est, max_features=mf, max_depth=md, learning_rate=lr, criterion='mse')
  GB.fit(X_train, y_train)

  # mse
  GB_mse = np.sqrt(mean_squared_error(y_test, GB.predict(X_test)))

  if visualFeatures:
    sns.barplot(GB.feature_importances_, X.columns)
    plt.show()

  return {'model':model, 'n_estimators':n_est, 'max_depth':md, 'max_features':mf, 'learning_rate':lr, 'mse':GB_mse}

  ### Hyperparameter(n_estimators)

In [74]:
GB_mse = pd.DataFrame(columns=['model','n_estimators','max_depth','max_features','learning_rate','mse'])
a = [50, 100, 200, 300, 500, 700, 1000]
index = 1

for i in a:
  model = 'GB-'+str(index)
  index += 1
  res = gradient(model, X, y, n_est=i )
  print(res)
  GB_mse = GB_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'GB-1', 'n_estimators': 50, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 180.0549896959866}
{'model': 'GB-2', 'n_estimators': 100, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 179.68942560458913}
{'model': 'GB-3', 'n_estimators': 200, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 178.97572096387782}
{'model': 'GB-4', 'n_estimators': 300, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 178.28520390832037}
{'model': 'GB-5', 'n_estimators': 500, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 176.9682758308195}
{'model': 'GB-6', 'n_estimators': 700, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 175.72948764154816}
{'model': 'GB-7', 'n_estimators': 1000, 'max_depth': None, 'max_features': 'auto', 'learning_rate': 0.0001, 'mse': 174.02009482796694}


  ### Hyperparameter(max_features)

In [76]:
a = [1,2,4,5,7]

for i in a :
  model = 'GB-'+str(index)
  index += 1
  res = gradient(model, X, y, n_est=1000 , mf=i)
  print(res)
  GB_mse = GB_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'GB-8', 'n_estimators': 1000, 'max_depth': None, 'max_features': 1, 'learning_rate': 0.0001, 'mse': 173.77593237384363}
{'model': 'GB-9', 'n_estimators': 1000, 'max_depth': None, 'max_features': 2, 'learning_rate': 0.0001, 'mse': 173.69957108792434}
{'model': 'GB-10', 'n_estimators': 1000, 'max_depth': None, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 173.62201384667526}
{'model': 'GB-11', 'n_estimators': 1000, 'max_depth': None, 'max_features': 5, 'learning_rate': 0.0001, 'mse': 173.6241299168518}
{'model': 'GB-12', 'n_estimators': 1000, 'max_depth': None, 'max_features': 7, 'learning_rate': 0.0001, 'mse': 174.02372140344548}


  ### Hyperparameter(max_depth)

In [78]:
a = [3,5,7,10,13,15]

for i in a:
  model = 'GB-'+str(index)
  index += 1
  res = gradient(model, X, y, n_est= 1000, mf=4 , md=i)
  print(res)
  GB_mse = GB_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'GB-13', 'n_estimators': 1000, 'max_depth': 3, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 176.73787898719416}
{'model': 'GB-14', 'n_estimators': 1000, 'max_depth': 5, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 175.78821026518668}
{'model': 'GB-15', 'n_estimators': 1000, 'max_depth': 7, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 175.15586316666608}
{'model': 'GB-16', 'n_estimators': 1000, 'max_depth': 10, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 174.4196933945583}
{'model': 'GB-17', 'n_estimators': 1000, 'max_depth': 13, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 173.9164992607649}
{'model': 'GB-18', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 173.76307271300666}


  ### Hyperparameter(learning_rate)

In [80]:
a = [0.0001, 0.001, 0.01, 0.1]

for i in a:
  model = 'GB-'+str(index)
  index += 1
  res = gradient(model, X, y, n_est=1000 , mf= 4, md=15 , lr=i)
  print(res)
  GB_mse = GB_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'GB-19', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.0001, 'mse': 173.7503959881156}
{'model': 'GB-20', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.001, 'mse': 152.48877264701636}
{'model': 'GB-21', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.01, 'mse': 166.31342841934168}
{'model': 'GB-22', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.1, 'mse': 168.32912104211016}


In [82]:
a = [0.3,0.5,0.7,0.9]

for i in a:
  model = 'GB-'+str(index)
  index += 1
  res = gradient(model, X, y, n_est=1000 , mf= 4, md=15 , lr=i)
  print(res)
  GB_mse = GB_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'GB-23', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.3, 'mse': 171.74929282549303}
{'model': 'GB-24', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.5, 'mse': 176.47313194069116}
{'model': 'GB-25', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.7, 'mse': 182.00393970967272}
{'model': 'GB-26', 'n_estimators': 1000, 'max_depth': 15, 'max_features': 4, 'learning_rate': 0.9, 'mse': 194.2547634244998}


In [83]:
GB_mse.sort_values(by='mse').head()

,model,n_estimators,max_depth,max_features,learning_rate,mse
19,GB-20,1000,15,4,0.0010,152.488773
20,GB-21,1000,15,4,0.0100,166.313428
21,GB-22,1000,15,4,0.1000,168.329121
22,GB-23,1000,15,4,0.3000,171.749293
9,GB-10,1000,None,4,0.0001,173.622014


## 7, LightGBM

### 모델링

In [84]:
from lightgbm import LGBMRegressor

def lightgbm(model,X,y, n_est, lr=0.0001, md=None):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify =y,test_size=0.3, random_state=2045)

  # modeling
  LG = LGBMRegressor(linear_tree = True, boosting_type='gbdt', objective ='regression',
                   n_estimators = n_est, learning_rate=lr, max_depth=md, n_jobs=-1)
  LG.fit(X_train, y_train)

  # mse
  LG_mse = np.sqrt(mean_squared_error(y_test, LG.predict(X_test)))



  return {'model':model, 'n_estimators':n_est, 'max_depth':md, 'learning_rate':lr, 'mse':LG_mse}

  ### Hyperparameter(n_estimators)

In [85]:
LG_mse = pd.DataFrame(columns=['model','n_estimators','max_depth','learning_rate','mse'])
a = [50, 100, 200, 300, 500, 700, 1000]
index = 1

for i in a:
  model = 'LG-'+str(index)
  index += 1
  res = lightgbm(model, X, y, n_est=i )
  print(res)
  LG_mse = LG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'LG-1', 'n_estimators': 50, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 180.15384201142484}
{'model': 'LG-2', 'n_estimators': 100, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 179.88454182819333}
{'model': 'LG-3', 'n_estimators': 200, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 179.35441742967794}
{'model': 'LG-4', 'n_estimators': 300, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 178.83542241592446}
{'model': 'LG-5', 'n_estimators': 500, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 177.81701047328326}
{'model': 'LG-6', 'n_estimators': 700, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 176.830145482395}
{'model': 'LG-7', 'n_estimators': 1000, 'max_depth': None, 'learning_rate': 0.0001, 'mse': 175.40838250259893}


  ### Hyperparameter(max_depth)

In [86]:
a = [3,5,7,10,13,15]

for i in a:
  model = 'LG-'+str(index)
  index += 1
  res = lightgbm(model, X, y, n_est= 1000, md=i)
  print(res)
  LG_mse = LG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'LG-8', 'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.0001, 'mse': 176.3102793557173}
{'model': 'LG-9', 'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.0001, 'mse': 175.51271177909922}
{'model': 'LG-10', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.0001, 'mse': 175.38604609330207}
{'model': 'LG-11', 'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.0001, 'mse': 175.40838250259893}
{'model': 'LG-12', 'n_estimators': 1000, 'max_depth': 13, 'learning_rate': 0.0001, 'mse': 175.40838250259893}
{'model': 'LG-13', 'n_estimators': 1000, 'max_depth': 15, 'learning_rate': 0.0001, 'mse': 175.40838250259893}


  ### Hyperparameter(learning_rate)

In [87]:
a = [0.001,0.003,0.005,0.007,0.009]

for i in a:
  model = 'LG-'+str(index)
  index += 1
  res = lightgbm(model, X, y, n_est=1000 , md=7, lr=i)
  print(res)
  LG_mse = LG_mse.append(res, ignore_index=True)
  DF_res = DF_res.append(res, ignore_index=True)

{'model': 'LG-14', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.001, 'mse': 153.4107542449748}
{'model': 'LG-15', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.003, 'mse': 146.50948162730805}
{'model': 'LG-16', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.005, 'mse': 145.8017632599479}
{'model': 'LG-17', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.007, 'mse': 145.9611088885786}
{'model': 'LG-18', 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.009, 'mse': 146.20936677092817}


## 성능비교

In [92]:
DF_res_sort = DF_res.sort_values(by='mse')

TypeError: ignored

In [ ]:
plt.figure(figsize = (15, 6))
sns.barplot(x = DF_res_sort.model, y = DF_res_sort['mse'])
plt.title('mse')
plt.show()